In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import scipy.stats as stats
from scipy.interpolate import interp1d
from scipy.optimize import minimize,least_squares,dual_annealing,differential_evolution
from datetime import datetime



In [2]:
def sabr_vector(F, alpha, beta, v, rho, K, T):
    """一个避免运行时警告的向量化版本"""
    # 确保K是numpy数组
    K = np.asarray(K)
    
    # 初始化结果数组
    bs_sigma = np.zeros_like(K, dtype=np.float64)
    
    # 创建一个布尔掩码来区分两种情况
    atm_mask = np.isclose(F, K, rtol=0, atol=1e-6)
    not_atm_mask = ~atm_mask

    # 1. 处理非ATM的情况
    if np.any(not_atm_mask):
        K_not_atm = K[not_atm_mask]
        z = v/alpha * (F*K_not_atm)**((1-beta)/2) * np.log(F/K_not_atm)
        xz = np.log( (np.sqrt(1-2*rho*z+z**2)+z-rho) / (1-rho) )
        
        first_term = alpha / ((F*K_not_atm)**((1-beta)/2) * (1 + (1-beta)**2/24 * np.log(F/K_not_atm)**2 + (1-beta)**4/1920 * np.log(F/K_not_atm)**4))
        second_term =  1 + T* ((1-beta)**2/24 * alpha**2 / (F*K_not_atm)**(1-beta) + 1/4* rho*beta*v*alpha / (F*K_not_atm)**((1-beta)/2) + (2-3*rho**2)/24*v**2)
        
        bs_sigma[not_atm_mask] = first_term * second_term * z/xz

    # 2. 处理ATM的情况
    if np.any(atm_mask):
        # 由于F和K接近，我们可以用F来代替K以简化计算
        fk_pow = F**(1 - beta)
        term1 = alpha / fk_pow
        term2 = (1 - beta)**2 * alpha**2 / (24 * fk_pow**2)
        term3 = rho * beta * v * alpha / (4 * fk_pow)
        term4 = (2 - 3 * rho**2) / 24 * v**2
        
        bs_sigma[atm_mask] = term1 * (1 + (term2 + term3 + term4) * T)
        
    return bs_sigma

In [3]:
def sabr_obloj(F,alpha,beta,v,rho,K,T):
    """
    F:当前远期价格
    alpha：F的波动率初始值
    beta：决定F的分布，可经验设置为0,0.5或1，也可当参数拟合
    v：波动率的波动率
    rho:相关系数
    K：行权价
    T；剩余期限
    obloj对hagan的解做了修正，对beta=0和F=K的特殊情况进行了改良，解决了beta趋近于1时的内部矛盾问题，优化了长期限和行权价格较小时的解的精度
    """
    #对于beta为1或者不为0的情况需要特别处理
    if beta == 1:
        z = v* np.log(F/K)/alpha
    else:
        z = v/alpha * (F**(1-beta)-K**(1-beta))/(1-beta)
    
    xz = np.log( (np.sqrt(1-2*rho*z+z**2)+z-rho) / (1-rho)  )

    first_term_F_eq_K = alpha / F**(1-beta)
    first_term_F_ineq_K = v * np.log(F/K) / xz
    first_term = np.where(np.isclose(F,K,rtol = 0,atol=0.1**6),first_term_F_eq_K, first_term_F_ineq_K)  #对于beta为1或者不为0的情况需要特别处理

    
    # second_term和SABR原文hagan解是保持一致的
    second_term =  1 + T* (                                                    \
                            (1-beta)**2/24 * alpha**2 / (F*K)**(1-beta)   +    \
                            1/4* rho*beta*v*alpha / (F*K)**((1-beta)/2)   +    \
                            (2-3*rho**2)/24*v**2                               \
                             )
    bs_sigma = first_term * second_term
    return bs_sigma


In [4]:
data = pd.read_csv('jd.csv', encoding='utf-8-sig')

In [5]:
data

,option_code,price,option_type,strike,time_to_expire
0,jd2508-C-3150.DF,389.5,call,3150,0.038356
1,jd2508-C-3200.DF,341.0,call,3200,0.038356
2,jd2508-C-3250.DF,293.5,call,3250,0.038356
3,jd2508-C-3300.DF,247.5,call,3300,0.038356
4,jd2508-C-3350.DF,204.0,call,3350,0.038356
...,...,...,...,...,...
83,jd2509-P-4100.DF,439.5,put,4100,0.128767
84,jd2509-P-4200.DF,533.0,put,4200,0.128767
85,jd2509-P-4300.DF,630.0,put,4300,0.128767
86,jd2509-P-4400.DF,728.5,put,4400,0.128767


In [19]:
data_raw = pd.read_pickle("data/sse50_option_data_processed_20231110.pkl")
data_raw = data_raw.sort_values(["contract_month","exerciseprice"])

In [4]:
data_raw

,date,tradecode,exerciseprice,close,settlement_price,S0,contract_month,CP,实虚值,last_day,maturity,r,F,q,market_imp_vol
80,2023-11-10,510050C2311M02250,2.25,0.2420,0.2420,2.49,2311,0,实值,2023-11-22,0.049587,0.0203,2.492693,-0.001496,NaN
82,2023-11-10,510050P2311M02250,2.25,0.0002,0.0002,2.49,2311,1,虚值,2023-11-22,0.049587,0.0203,2.492693,-0.001496,0.184617
81,2023-11-10,510050C2311M02300,2.30,0.1912,0.1912,2.49,2311,0,实值,2023-11-22,0.049587,0.0203,2.492693,-0.001496,NaN
83,2023-11-10,510050P2311M02300,2.30,0.0003,0.0003,2.49,2311,1,虚值,2023-11-22,0.049587,0.0203,2.492693,-0.001496,0.156664
78,2023-11-10,510050C2311M02350,2.35,0.1433,0.1433,2.49,2311,0,实值,2023-11-22,0.049587,0.0203,2.492693,-0.001496,0.137044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2023-11-10,510050P2406A02650,2.65,0.1887,0.1887,2.49,2406,1,实值,2024-06-26,0.946281,0.0203,2.542101,-0.001584,0.130555
102,2023-11-10,510050C2406A02700,2.70,0.0683,0.0683,2.49,2406,0,虚值,2024-06-26,0.946281,0.0203,2.542101,-0.001584,0.131906
103,2023-11-10,510050P2406A02700,2.70,0.2245,0.2240,2.49,2406,1,实值,2024-06-26,0.946281,0.0203,2.542101,-0.001584,0.133368
104,2023-11-10,510050C2406A02750,2.75,0.0556,0.0556,2.49,2406,0,虚值,2024-06-26,0.946281,0.0203,2.542101,-0.001584,0.132710


xtquant文档地址：http://dict.thinktrader.net/nativeApi/start_now.html
